## 8 Puzzle Solver

Group 1

Tutorial Group: FCS7

Team Members: Tee Wei Ping, Sih Jia Qi, Ponyuenyong Kritchanat

Link to Google Drive containing the Interactive Puzzle Gameplay & Video Presentation: https://drive.google.com/drive/folders/1g6w9Z6xI8YkzBtKzhFsX4ZCcpffFxEUB?usp=sharing

## 8 Puzzle Solver Introduction:

The 8-puzzle solver is a classic single-player puzzle game that challenges players to rearrange numbered tiles on a 3x3 grid to reach a specific configuration. The objective of the game is to reach the goal state by sliding tiles into the empty space, ultimately forming a predefined pattern. Players use logical deduction and strategy to move tiles while avoiding configurations that lead to unsolvability.

## State Representation
- The state is represented as a single number starting from first row and first column as the most significant digit, and the bottom right as least significant digit, so the following state is represented as the number "102345678"
<table align="center">
  <tr>
    <td>1</td>
    <td>0</td>
    <td>2</td>
  </tr>
  <tr>
    <td>3</td>
    <td>4</td>
    <td>5</td>
  </tr>
  <tr>
    <td>6</td>
    <td>7</td>
    <td>8</td>
  </tr>
</table>

- The state is stored as integer to reduce the amount of space the application uses, integer is stored in 4 bytes while storing the state as string would cost 9 bytes.
- The state is then converted to a string to be easily proccessed.
- Getting string representation of a state is done through a simple function
  ```python
  def getStringRepresentation(x):
      if(int(math.log10(x))+1 == 9):
          return str(x)
      else :
          return "0"+str(x)   
  ```
- Getting the next possible states is done throught a simple algorithm which goes as follows
  * Convert the index of character "0" in state to 2D index
    ```python
    idx = state.index('0')
    i = int(idx / 3)
    j = int(idx % 3)
    ```
  * Get the possible moves in all 4 directions
    ```python
    dx = [-1, 1, 0, 0]
    dy = [0, 0, 1, -1]
    for x in range(0, 4):
        nx = i + dx[x]
        ny = j + dy[x]
    ```
  * Check if the new 2D index is a valid index 
    ```python
    def checkValid(nx, ny):
    if nx >= 3 or nx < 0 or ny >= 3 or ny < 0:
        return 0
    return 1
    ```
  * Convert the index of possible moves to 1D 
    ```python
    nwIdx = int(nx * 3 + ny)
    ```
  * The next state is a new string where charachter "0" and the character at the new index are swapped

## Analysis for Different Algorithms.
- For the following random test case:
<table align="center">
  <tr>
    <td>7</td>
    <td>0</td>
    <td>2</td>
  </tr>
  <tr>
    <td>8</td>
    <td>5</td>
    <td>3</td>
  </tr>
  <tr>
    <td>6</td>
    <td>1</td>
    <td>4</td>
  </tr>
</table>
<br>
<table align="center">
  <tr>
    <th>Algorithm</th>
    <th>Cost of Path</th>
    <th>Nodes Expanded</th>
    <th>Search Depth</th>
    <th>Running time</th>
  </tr>
  <tr>
    <td>Depth-First Search</td>
    <td>54497</td>
    <td>63397</td>
    <td>54497</td>
    <td>0.22s</td>
  </tr>
  <tr>
    <td>Breadth-First Search</td>
    <td>27</td>
    <td>174386</td>
    <td>27</td>
    <td>6.15s</td>
  </tr>
  <tr>
    <td>A* Search</td>
    <td>27</td>
    <td>3495</td>
    <td>27</td>
    <td>0.05s</td>
  </tr>
  <tr>
    <td>Uniform-Cost Search</td>
    <td>27</td>
    <td>170995</td>
    <td>27</td>
    <td>0.66s</td>
  </tr>
  <tr>
    <td>Depth-Limited Search</td>
    <td>99</td>
    <td>75288</td>
    <td>99</td>
    <td>0.14s</td>
  </tr>
  <tr>
    <td>Iterative Deepening Search</td>
    <td>Unsolvable (Solution not found within depth limit)</td>
    <td>Unsolvable (Solution not found within depth limit)</td>
    <td>Unsolvable (Solution not found within depth limit)</td>
    <td>Unsolvable (Solution not found within depth limit)</td>
  </tr>
  <tr>
    <td>Greedy Search</td>
    <td>71</td>
    <td>303</td>
    <td>-</td>
    <td>0.002s</td>
  </tr>
</table>

In [2]:
import heapq
import math
import time

dx = [-1, 1, 0, 0]
dy = [0, 0, 1, -1]

dfs_counter = 0
bfs_counter = 0
astar_counter = 0
ucs_counter = 0
dls_counter = 0
ids_counter = 0
greedy_counter = 0

dfs_path = []
bfs_path = []
astar_path = []
ucs_path = []
dls_path = []
ids_path = []
greedy_path = []

dfs_cost = 0
bfs_cost = 0
astar_cost = 0
ucs_cost = 0
dls_cost = 0
ids_cost = 0
greedy_cost = 0

dfs_depth = 0
bfs_depth = 0
astar_depth = 0
ucs_depth = 0
dls_depth = 0
ids_depth = 0
greedy_depth = 0

time_dfs = 0
time_bfs = 0
time_astar = 0
time_ucs = 0
time_dls = 0
time_ids = 0
time_greedy = 0


def getStringRepresentation(x):
    if int(math.log10(x)) + 1 == 9:
        return str(x)
    else:
        return "0" + str(x)


def getChildren(state):
    children = []
    idx = state.index('0')
    i = int(idx / 3)
    j = int(idx % 3)
    for x in range(0, 4):
        nx = i + dx[x]
        ny = j + dy[x]
        nwIdx = int(nx * 3 + ny)
        if checkValid(nx, ny):
            listTemp = list(state)
            listTemp[idx], listTemp[nwIdx] = listTemp[nwIdx], listTemp[idx]
            children.append(''.join(listTemp))
    return children


def getPath(parentMap, inputState):
    path = []
    temp = 12345678
    while temp != inputState:
        path.append(temp)
        temp = parentMap[temp]
    path.append(inputState)
    path.reverse()
    return path


def printPath(path):
    for i in path:
        print(getStringRepresentation(i))


def goalTest(state):
    if state == 12345678:
        return True
    return False


def isSolvable(digit):
    count = 0
    for i in range(0, 9):
        for j in range(i, 9):
            if digit[i] > digit[j] and digit[i] != 9:
                count += 1
    return count % 2 == 0


def BFS(inputState):
    start_time = time.time()
    q = []
    explored = {}
    parent = {}
    parent_cost = {}
    integer_state = int(inputState)
    q.append(integer_state)
    cnt = 0
    global bfs_counter
    global bfs_path
    global bfs_cost
    global bfs_depth
    global time_bfs
    bfs_depth = 0
    parent_cost[integer_state] = 0
    while q:
        cnt += 1
        state = q.pop(0)
        explored[state] = 1
        bfs_depth = max(bfs_depth, parent_cost[state])
        if goalTest(state):
            path = getPath(parent, int(inputState))
            bfs_counter = cnt
            bfs_path = path
            bfs_cost = len(path) - 1
            time_bfs = float(time.time() - start_time)
            print("BFS Path: ", bfs_path)
            print("BFS Cost: ", bfs_cost)
            print("BFS Nodes Expanded: ", bfs_counter)
            print("BFS Depth: ", bfs_depth)
            print("BFS Time: ", time_bfs)
            return 1
        children = getChildren(getStringRepresentation(state))
        for child in children:
            child_int = int(child)
            if child_int not in explored:
                q.append(child_int)
                parent[child_int] = state
                explored[child_int] = 1
                parent_cost[child_int] = 1 + parent_cost[state]
    bfs_path = []
    bfs_cost = 0
    bfs_counter = cnt
    time_bfs = float(time.time() - start_time)
    print("Unsolvable")
    return 0


def DFS(inputState):
    start_time = time.time()
    stack = []
    explored = {}
    parent = {}
    parent_cost = {}
    integer_state = int(inputState)
    parent_cost[integer_state] = 0
    explored[integer_state] = 1
    stack.append(integer_state)
    cnt = 0
    global dfs_counter
    global dfs_path
    global dfs_cost
    global dfs_depth
    global time_dfs
    dfs_depth = 0
    while stack:
        cnt += 1
        state = stack[-1]
        stack.pop()
        dfs_depth = max(dfs_depth, parent_cost[state])
        if goalTest(state):
            path = getPath(parent, int(inputState))
            dfs_counter = cnt
            dfs_path = path
            dfs_cost = len(path) - 1
            time_dfs = float(time.time() - start_time)
            print("DFS Path: ", dfs_path)
            print("DFS Cost: ", dfs_cost)
            print("DFS Nodes Expanded: ", dfs_counter)
            print("DFS Depth: ", dfs_depth)
            print("DFS Time: ", time_dfs)
            return 1
        children = getChildren(getStringRepresentation(state))
        for child in children:
            child_int = int(child)
            if child_int not in explored:
                stack.append(child_int)
                parent[child_int] = state
                explored[child_int] = 1
                parent_cost[child_int] = 1 + parent_cost[state]
    dfs_path = []
    dfs_cost = 0
    dfs_counter = cnt
    time_dfs = float(time.time() - start_time)
    print("Unsolvable")
    return 0


def checkValid(i, j):
    if i >= 3 or i < 0 or j >= 3 or j < 0:
        return 0
    return 1


def getDistance(state):
    tot = 0
    for i in range(1, 9):
        goalX = int(i / 3)
        goalY = i % 3
        idx = state.index(str(i))
        itemX = int(idx / 3)
        itemY = idx % 3
        tot += (abs(goalX - itemX) + abs(goalY - itemY))
    return tot


def AStarSearch(inputState):
    start_time = time.time()
    integer_state = int(inputState)
    heap = []
    explored = {}
    parent = {}
    cost_map = {}
    heapq.heappush(heap, (getDistance(inputState), integer_state))
    cost_map[integer_state] = getDistance(inputState)
    heap_map = {}
    heap_map[integer_state] = 1
    global astar_counter
    global astar_path
    global astar_cost
    global astar_depth
    global time_astar
    astar_depth = 0
    while heap:
        node = heapq.heappop(heap)
        state = node[1]
        string_state = getStringRepresentation(state)
        parent_cost = node[0] - getDistance(string_state)
        if not state in explored:
            astar_depth = max(parent_cost, astar_depth)
        explored[state] = 1

        if goalTest(state):
            path = getPath(parent, int(inputState))
            astar_path = path
            astar_counter = (len(explored))
            astar_cost = len(path) - 1
            time_astar = float(time.time() - start_time)
            print("A* Path: ", astar_path)
            print("A* Cost: ", astar_cost)
            print("A* Nodes Expanded: ", astar_counter)
            print("A* Depth: ", astar_depth)
            print("A* Time: ", time_astar)
            return 1

        children = getChildren(string_state)
        for child in children:
            new_cost = getDistance(child)
            child_int = int(child)
            if child_int not in explored and child not in heap_map:
                heapq.heappush(heap, (parent_cost + new_cost + 1, child_int))
                heap_map[child_int] = 1
                cost_map[child_int] = parent_cost + new_cost + 1
                parent[child_int] = state
            elif child_int in heap_map:
                if (new_cost + parent_cost + 1) < cost_map[child_int]:
                    parent[child_int] = state
                    cost_map[child_int] = new_cost + parent_cost + 1
                    heapq.heappush(heap, (parent_cost + 1 + new_cost, child_int))
    astar_cost = 0
    astar_path = []
    astar_counter = (len(explored))
    time_astar = float(time.time() - start_time)
    print("Unsolvable")
    return 0


def uniform_cost_search(inputState):
    start_time = time.time()

    integer_state = int(inputState)
    
    frontier = [(0, integer_state)]
    came_from = {integer_state: None}
    cost_so_far = {integer_state: 0}

    global ucs_counter, ucs_path, ucs_cost, ucs_depth, time_ucs
    ucs_counter = 0
    ucs_depth = 0

    while frontier:
        current_cost, current_state = heapq.heappop(frontier)

        ucs_counter += 1

        if goalTest(current_state):
            ucs_path = getPath(came_from, integer_state)
            ucs_cost = len(ucs_path) - 1
            ucs_depth = max(ucs_depth, cost_so_far[current_state])
            time_ucs = time.time() - start_time
            print("UCS Path: ", ucs_path)
            print("UCS Cost: ", ucs_cost)
            print("UCS Nodes Expanded: ", ucs_counter)
            print("UCS Depth: ", ucs_depth)
            print("UCS Time: ", time_ucs)
            return 1

        for next_state_str in getChildren(getStringRepresentation(current_state)):
            next_state = int(next_state_str)
            new_cost = cost_so_far[current_state] + 1
            if next_state not in cost_so_far or new_cost < cost_so_far[next_state]:
                cost_so_far[next_state] = new_cost
                priority = new_cost
                heapq.heappush(frontier, (priority, next_state))
                came_from[next_state] = current_state

    ucs_path = []
    ucs_cost = 0
    time_ucs = time.time() - start_time
    print("Unsolvable")
    return 0


def depth_limited_search(inputState, limit=100):
    global dls_counter, dls_path, dls_cost, dls_depth, time_dls
    start_time = time.time()
    
    stack = [(inputState, 0)]
    parentMap = {inputState: None}
    visited = set([inputState])
    dls_counter = 0
    found = False

    while stack:
        state, depth = stack.pop()
        dls_counter += 1
        
        if goalTest(int(state)):
            found = True
            # Reconstruct the path
            dls_path = []
            while state is not None:
                dls_path.append(int(state))
                state = parentMap.get(str(state))
            dls_path.reverse()
            dls_cost = len(dls_path) - 1
            dls_depth = depth
            break
        
        if depth < limit:
            for child in getChildren(str(state)):
                if child not in visited:
                    visited.add(child)
                    parentMap[child] = str(state)
                    stack.append((child, depth + 1))

    time_dls = time.time() - start_time

    if found:
        print("DLS Path: ", dls_path)
        print("DLS Cost: ", dls_cost)
        print("DLS Nodes Expanded: ", dls_counter)
        print("DLS Depth: ", dls_depth)
        print("DLS Time: ", time_dls)
        return 1
    else:
        print("Unsolvable")
        return 0


def iterative_deepening_search(inputState):
    global ids_counter, ids_path, ids_cost, ids_depth, time_ids
    start_time = time.time()
    
    ids_counter = 0
    found = False
    depth_limit = 0
    MAX_DEPTH = 20
    
    while not found and depth_limit <= MAX_DEPTH:
        stack = [(inputState, 0)]
        visited = set([inputState])
        
        while stack:
            current_state, current_depth = stack.pop()
            ids_counter += 1
            
            if goalTest(current_state):
                found = True
                ids_path = [current_state]
                ids_cost = current_depth
                ids_depth = current_depth
                break
            
            if current_depth < depth_limit:
                for child in getChildren(current_state):
                    if child not in visited:
                        visited.add(child)
                        stack.append((child, current_depth + 1))
        
        depth_limit += 1
    
    time_ids = time.time() - start_time

    if found:
        print("IDS Path: ", ids_path)
        print("IDS Cost: ", ids_cost)
        print("IDS Nodes Expanded: ", ids_counter)
        print("IDS Depth: ", ids_depth)
        print("IDS Time: ", time_ids)
        return 1
    else:
        print("Unsolvable")
        return 0


def greedy_search(inputState):
    start_time = time.time()

    integer_state = int(inputState) if isinstance(inputState, str) else inputState
    
    frontier = [(getDistance(getStringRepresentation(integer_state)), integer_state)]
    came_from = {integer_state: None}
    
    global greedy_counter, greedy_path, greedy_cost, greedy_depth, time_greedy
    greedy_counter = 0
    greedy_depth = 0

    while frontier:
        current_heuristic, current_state = heapq.heappop(frontier)

        greedy_counter += 1

        if goalTest(current_state):
            greedy_path = getPath(came_from, integer_state)
            greedy_cost = len(greedy_path) - 1
            time_greedy = time.time() - start_time
            print("Greedy Path: ", greedy_path)
            print("Greedy Cost: ", greedy_cost)
            print("Greedy Nodes Expanded: ", greedy_counter)
            print("Greedy Depth: ", greedy_depth)
            print("Greedy Time: ", time_greedy)
            return 1

        for next_state_str in getChildren(getStringRepresentation(current_state)):
            next_state = int(next_state_str)
            if next_state not in came_from:
                heapq.heappush(frontier, (getDistance(getStringRepresentation(next_state)), next_state))
                came_from[next_state] = current_state

    time_greedy = time.time() - start_time
    print("Unsolvable")
    return 0

In [3]:
DFS('702853614')

DFS Path:  [702853614, 72853614, 872053614, 872503614, 872530614, 872534610, 872534601, 872534061, 872034561, 872304561, 872340561, 872341560, 872341506, 872341056, 872041356, 872401356, 872410356, 872416350, 872416305, 872416035, 872016435, 872106435, 872160435, 872165430, 872165403, 872165043, 872065143, 872605143, 872650143, 872653140, 872653104, 872603154, 872063154, 872163054, 872163504, 872163540, 872160543, 872106543, 872016543, 872516043, 872516403, 872516430, 872510436, 872501436, 872051436, 72851436, 702851436, 720851436, 721850436, 721805436, 721085436, 721485036, 721485306, 721485360, 721480365, 721408365, 721048365, 721348065, 721348605, 721348650, 721340658, 721304658, 721034658, 721634058, 721634508, 721634580, 721630584, 721603584, 721063584, 721563084, 721563804, 721563840, 721560843, 721506843, 721056843, 721856043, 721856403, 721806453, 721086453, 721486053, 721486503, 721486530, 721480536, 721408536, 721048536, 721548036, 721548306, 721548360, 721540368, 721504368, 

1

In [4]:
BFS('702853614')

BFS Path:  [702853614, 752803614, 752830614, 750832614, 705832614, 735802614, 735812604, 735812064, 735012864, 35712864, 305712864, 315702864, 315762804, 315762084, 315062784, 315602784, 315620784, 315624780, 315624708, 315624078, 315024678, 15324678, 105324678, 125304678, 125340678, 120345678, 102345678, 12345678]
BFS Cost:  27
BFS Nodes Expanded:  174386
BFS Depth:  27
BFS Time:  6.148829698562622


1

In [5]:
AStarSearch('702853614')

A* Path:  [702853614, 752803614, 752830614, 750832614, 705832614, 735802614, 735812604, 735812064, 735012864, 35712864, 305712864, 315702864, 315762804, 315762084, 315062784, 315602784, 315620784, 315624780, 315624708, 315624078, 315024678, 15324678, 105324678, 125304678, 125340678, 120345678, 102345678, 12345678]
A* Cost:  27
A* Nodes Expanded:  3495
A* Depth:  27
A* Time:  0.05101418495178223


1

In [6]:
uniform_cost_search('702853614')

UCS Path:  [702853614, 752803614, 752830614, 750832614, 705832614, 735802614, 735812604, 735812064, 735012864, 35712864, 305712864, 315702864, 315762804, 315762084, 315062784, 315602784, 315620784, 315624780, 315624708, 315624078, 315024678, 15324678, 105324678, 125304678, 125340678, 120345678, 102345678, 12345678]
UCS Cost:  27
UCS Nodes Expanded:  170995
UCS Depth:  27
UCS Time:  0.6577203273773193


1

In [7]:
depth_limited_search('702853614')

DLS Path:  [702853614, 72853614, 872053614, 872503614, 872530614, 872534610, 872534601, 872534061, 872034561, 872304561, 872340561, 872341560, 872341506, 872341056, 872041356, 872401356, 872410356, 872416350, 872416305, 872416035, 872016435, 872106435, 872160435, 872165430, 872165403, 872165043, 872065143, 872605143, 872650143, 872653140, 872653104, 872603154, 872063154, 872163054, 872163504, 872163540, 872160543, 872106543, 872016543, 872516043, 872516403, 872516430, 872510436, 872501436, 872051436, 72851436, 702851436, 720851436, 721850436, 721805436, 721085436, 721485036, 721485306, 721485360, 721480365, 721408365, 721048365, 721348065, 721348605, 721348650, 721340658, 721304658, 721034658, 21734658, 201734658, 231704658, 231074658, 31274658, 301274658, 310274658, 314270658, 314207658, 314027658, 314627058, 314627508, 314627580, 314620587, 314602587, 314062587, 314562087, 314562807, 314502867, 314052867, 314852067, 314852607, 314802657, 314082657, 14382657, 104382657, 140382657, 142

1

In [8]:
iterative_deepening_search('102345678')

Unsolvable


0

In [9]:
greedy_search('702853614')

Greedy Path:  [702853614, 752803614, 752083614, 52783614, 502783614, 582703614, 582713604, 582713640, 582710643, 580712643, 508712643, 518702643, 518072643, 18572643, 108572643, 178502643, 178542603, 178542630, 178540632, 170548632, 107548632, 147508632, 147538602, 147538620, 147530628, 140537628, 104537628, 14537628, 514037628, 514307628, 514370628, 510374628, 501374628, 51374628, 351074628, 351704628, 301754628, 310754628, 314750628, 314705628, 314075628, 14375628, 104375628, 140375628, 145370628, 145307628, 145327608, 145327680, 145320687, 140325687, 104325687, 124305687, 124385607, 124385670, 124380675, 120384675, 102384675, 182304675, 182340675, 182345670, 182345607, 182305647, 102385647, 120385647, 125380647, 125308647, 125348607, 125348670, 125340678, 120345678, 102345678, 12345678]
Greedy Cost:  71
Greedy Nodes Expanded:  303
Greedy Depth:  0
Greedy Time:  0.002001523971557617


1